In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

2023-04-17 05:37:30.618232: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 05:37:30.813909: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-17 05:37:30.816506: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 05:37:31.732249: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset_path = os.listdir('MangoLeafBD')
print(dataset_path)

['Sooty Mould', 'Powdery Mildew', 'Gall Midge', 'Cutting Weevil', 'Die Back', 'Bacterial Canker', 'Anthracnose', 'Healthy']


In [3]:
class_labels = []

for item in dataset_path:
 # Get all the file names
    all_classes = os.listdir('MangoLeafBD' + '/' +item)
 #print(all_classes)

 # Add them to the list
    for room in all_classes:
        class_labels.append((item, str('dataset_path' + '/' +item) + '/' + room))

In [4]:
df = pd.DataFrame(data=class_labels, columns=['Labels', 'image'])
print(df.head())

        Labels                                              image
0  Sooty Mould  dataset_path/Sooty Mould/IMG_20211108_121559 (...
1  Sooty Mould  dataset_path/Sooty Mould/IMG_20211211_154825 (...
2  Sooty Mould  dataset_path/Sooty Mould/IMG_20211211_154608 (...
3  Sooty Mould  dataset_path/Sooty Mould/IMG_20211211_154225 (...
4  Sooty Mould  dataset_path/Sooty Mould/IMG_20211211_154252 (...


In [5]:
print(df.tail())

       Labels                                              image
3995  Healthy  dataset_path/Healthy/20211231_155753 (Custom).jpg
3996  Healthy  dataset_path/Healthy/20211231_125054 (Custom).jpg
3997  Healthy  dataset_path/Healthy/20211231_161108 (Custom).jpg
3998  Healthy  dataset_path/Healthy/20211231_124957 (Custom).jpg
3999  Healthy  dataset_path/Healthy/20211231_160229 (Custom).jpg


In [6]:
print(df['Labels'].value_counts())

Sooty Mould         500
Powdery Mildew      500
Gall Midge          500
Cutting Weevil      500
Die Back            500
Bacterial Canker    500
Anthracnose         500
Healthy             500
Name: Labels, dtype: int64


In [7]:
path = 'MangoLeafBD/'

im_size = 224

images = []
labels = []

for i in dataset_path:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]
    
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (im_size, im_size))
        images.append(img)
        labels.append(i)

In [8]:
images = np.array(images)
images = images.astype('float32') / 255.0
images.shape

(4000, 224, 224, 3)

In [14]:
from sklearn.preprocessing import LabelEncoder , OneHotEncoder
import sys
np.set_printoptions(threshold=sys.maxsize)
y=df['Labels'].values
# print(y)
for i in range(0,len(y), 470):
    print(y[i])
y_labelencoder = LabelEncoder ()
y = y_labelencoder.fit_transform (y)
# print (y)
for i in range(0,len(y), 470):
    print(y[i])

Sooty Mould
Sooty Mould
Powdery Mildew
Gall Midge
Cutting Weevil
Die Back
Bacterial Canker
Anthracnose
Healthy
7
7
6
4
2
3
1
0
5


In [ ]:
y=y.reshape(-1,1)

from sklearn.compose import ColumnTransformer
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(y).toarray()


In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


images, Y = shuffle(images, Y, random_state=1)


train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=415)

#inpect the shape of the training and testing.
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
from matplotlib.pyplot import imread
from matplotlib.pyplot import imshow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG19

In [ ]:
NUM_CLASSES = 8
IMG_SIZE = 224

In [ ]:
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    #x = img_augmentation(inputs)
    x = inputs
    model = VGG19(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.Flatten()(model.output)
#     x = layers.BatchNormalization()(x)

#     top_dropout_rate = 0.2
#     x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="VGG19")
#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer='adam', loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=NUM_CLASSES)


In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    #plt.plot(hist.history["val_accuracy"])
    plt.title("VGG19 Accuracy")
    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend(["train", "validation"], loc="upper left")
    plt.grid()
    plt.savefig('VGG19_acc',dpi=300)
    plt.show()

In [ ]:
epochs = 5


hist = model.fit(train_x, train_y, epochs=epochs, verbose=2)
    

In [ ]:
plot_hist(hist)

In [ ]:
preds = model.evaluate(test_x, test_y)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
import pickle

In [ ]:
with open('VGG19History','wb') as f:
    pickle.dump(hist.history,f)

In [ ]:
df = pd.DataFrame.from_dict(hist.history)
df.to_csv('VGG19_history.csv')

In [ ]:
model.save('VGG19_detect.h5')